In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Import trading libraries for live/paper trading (Alpaca, IB, etc.)
from alpaca_trade_api import REST

# Set up Alpaca (replace with your credentials)
API_KEY = 'PKUYV5B3MGE6RAYVN71B'
API_SECRET = 'PKUYV5B3MGE6RAYVN71B'
BASE_URL = 'https://paper-api.alpaca.markets/v2'
alpaca = REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')


# Function to fetch AMD stock data
def fetch_stock_data(ticker, start_date, end_date):
    stock = yf.download(ticker, start=start_date, end=end_date)
    return stock


# Function to fetch AMD options data
def fetch_options_data(ticker, expiration_date):
    stock = yf.Ticker(ticker)
    options = stock.option_chain(expiration_date)
    return options.calls


# Covered Call Strategy
def execute_covered_call(stock_ticker, budget, expiration_date, delta_target=0.1, target_premium=0.01):
    # Fetch stock price
    stock_data = fetch_stock_data(stock_ticker, (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d'), datetime.now().strftime('%Y-%m-%d'))
    latest_price = float(stock_data['Close'].iloc[-1])

    # Fetch options data
    options_data = fetch_options_data(stock_ticker, expiration_date)

    options_data['Delta'] = options_data['impliedVolatility'] * latest_price  # Placeholder delta approximation

    # Filter options by delta and premium
    filtered_options = options_data[
        (options_data['Delta'].astype(float) >= float(delta_target)) & 
        (options_data['lastPrice'].astype(float) >= float(target_premium * latest_price))
    ]

    if not filtered_options.empty:
        best_option = filtered_options.iloc[0]  # Choose the best option based on criteria
        print(f"Selected Option: {best_option['contractSymbol']}, Strike: {best_option['strike']}, Premium: {best_option['lastPrice']}")
        
        # Calculate shares and trade cost
        shares_to_buy = budget // latest_price
        trade_cost = shares_to_buy * latest_price

        print(f"Buying {shares_to_buy} shares of {stock_ticker} at ${latest_price} each, total cost: ${trade_cost}")

        # Sell calls
        print(f"Selling {shares_to_buy // 100} covered calls at strike {best_option['strike']}, premium ${best_option['lastPrice']} each.")
    else:
        print("No suitable options found based on the given criteria.")

def next_friday(start_date):
    days_ahead = 4 - start_date.weekday()  # 4 represents Friday (Monday=0, ..., Sunday=6)
    if days_ahead < 0:  # If it's already Friday or past Friday, go to the next week
        days_ahead += 7
    return start_date + timedelta(days=days_ahead)

# Example: Run strategy
if __name__ == '__main__':
    STOCK_TICKER = 'AMD'
    BUDGET = 30000  # Example budget
    # Find the closest Friday
    # Update EXPIRATION_DATE to the next Friday
    EXPIRATION_DATE = next_friday(datetime.now()+ timedelta(days=14)).strftime('%Y-%m-%d')
    execute_covered_call(STOCK_TICKER, BUDGET, EXPIRATION_DATE)


[*********************100%***********************]  1 of 1 completed
C:\Users\C70636\AppData\Local\Temp\ipykernel_1388\2499197885.py:34: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_price = float(stock_data['Close'].iloc[-1])


122.83999633789062
Selected Option: AMD250214C00065000, Strike: 65.0, Premium: 49.05
Buying 244.0 shares of AMD at $122.83999633789062 each, total cost: $29972.959106445312
Selling 2.0 covered calls at strike 65.0, premium $49.05 each.
